### FAST Algorithm for Corner Detection
All the above feature detection methods are good in some way. But they are not fast enough to work in real-time applications like SLAM. There comes the FAST algorithm, which is really "FAST".

#### 목표
In this chapter,

* We will understand the basics of FAST algorithm
* We will find corners using OpenCV functionalities for FAST algorithm.

#### 이론
몇 가지 기능 감지기를 보았고 그 중 많은 것이 정말 좋습니다. 그러나 실시간 응용 관점에서 보면 충분히 빠르지 않습니다. 하나의 가장 좋은 예는 계산 자원이 제한적인 SLAM (Simultaneous Localization and Mapping) 모바일 로봇입니다.

이를 해결하기 위해 2006 년에 Edward Rosten과 Tom Drummond가 "고속 코너 감지를위한 기계 학습"이라는 논문에서 FAST (Accelerated Segment Test의 특징) 알고리즘을 제안했습니다 (나중에 개정 됨). 알고리즘의 기본 요약은 다음과 같습니다. 자세한 내용은 원본 용지를 참조하십시오 (모든 이미지는 원본 용지에서 가져옴).

##### Feature Detection using FAST

1. 관심 지점으로 식별 될 이미지에서 픽셀 \\(p를 선택합니다. 그 강도를 I_p\\)로 합시다.
2. 적절한 임계 값 \\(t\\)를 선택하십시오.
3. 테스트중인 픽셀 주위에 16 픽셀의 원을 생각해보십시오. (아래 이미지 참조)
![](fast_speedtest.jpg)
4. 이제 \\(I_p + t보다 더 밝거나 I_p - t\\)보다 어두운 모든 원(16 픽셀)에 n연속 픽셀 세트가있는 경우 픽셀 p가 모퉁이입니다. (위 이미지에서 흰색 점선으로 표시). n은 12로 선택되었습니다.
5. 다수의 비 구석을 배제하기 위해 고속 테스트가 제안되었습니다. 이 테스트에서는 1, 9, 5 및 13의 네 픽셀 만 검사합니다 (처음 1과 9는 너무 밝거나 어두우면 테스트됩니다. 그렇다면 5와 13을 확인하십시오). p가 구석이라면 적어도 세 개는 모두 \\(I_p + t보다 밝거나 I_p-t\\)보다 어두워 야합니다. 두 경우 모두 해당하지 않으면 p가 구석이 될 수 없습니다. 그런 다음 원의 모든 픽셀을 검사하여 전체 세그먼트 테스트 기준을 전달 된 후보에 적용 할 수 있습니다. 이 검출기는 그 자체로 고성능을 나타내지 만 몇 가지 단점이 있습니다.

* 그것은 n <12에 대한 많은 후보자를 거부하지 않습니다.
* 픽셀의 선택은 질문의 순서 및 모서리 모양의 분포에 따라 효율이 달라 지므로 최적이 아닙니다.
* 고속 테스트 결과가 폐기됩니다.
* 여러 기능이 서로 인접하여 감지됩니다.

처음 3 점은 기계 학습 접근 방식으로 해결됩니다. 마지막 것은 최대가 아닌 억제를 사용하여 해결됩니다.

##### Machine Learning a Corner Detector
1. Select a set of images for training (preferably from the target application domain)
2. Run FAST algorithm in every images to find feature points.
3. For every feature point, store the 16 pixels around it as a vector. Do it for all the images to get feature vector P.
4. Each pixel (say x) in these 16 pixels can have one of the following three states:
~[](fast_eqns.jpg)
5. Depending on these states, the feature vector \\(P\\) is subdivided into 3 subsets, \\(P_d, P_s, P_b\\).
6. Define a new boolean variable, \\(K_p\\), which is true if \\(p\\) is a corner and false otherwise.
7. 진정한 클래스에 대한 지식을 위해 변수 \\(K_p\\)를 사용하여 각 하위 집합을 쿼리하려면 ID3 알고리즘 (의사 결정 트리 분류 자)을 사용하십시오. 후보 픽셀이 \\(K_p\\)의 엔트로피로 측정 된 구석인지 여부에 대한 대부분의 정보를 산출하는 \\(x\\)를 선택합니다.
8. This is recursively applied to all the subsets until its entropy is zero.
9. The decision tree so created is used for fast detection in other images.

##### Non-maximal Suppression
인접한 장소에서 여러 관심 지점을 감지하는 것도 또 다른 문제입니다. 비 최대 억제를 사용하여 해결됩니다.

1. 검출 된 모든 특징점에 대해 스코어 함수 \\(V\\)를 계산합니다. \\(V는 p\\)와 주변 픽셀 값의 절대 차이의 합입니다.
2. 두 개의 인접한 키포인트를 고려하고 \\(V\\) 값을 계산하십시오.
3. \\(V\\) 값이 낮은 것을 폐기하십시오.

##### Summary
It is several times faster than other existing corner detectors.

But it is not robust to high levels of noise. It is dependent on a threshold.

#### FAST Feature Detector in OpenCV
그것은 OpenCV의 다른 어떤 특징 검출기라고 불립니다. 원하는 경우 임계 값, 적용 할 최대가 아닌 억제 여부, 사용할 인접 영역 등을 지정할 수 있습니다.

이웃에 대해 세 개의 플래그가 정의됩니다 (cv.FAST_FEATURE_DETECTOR_TYPE_5_8, cv.FAST_FEATURE_DETECTOR_TYPE_7_12 및 cv.FAST_FEATURE_DETECTOR_TYPE_9_16). 다음은 FAST 피쳐 포인트를 감지하고 그려내는 방법에 대한 간단한 코드입니다.

In [1]:
import numpy as np
import cv2 as cv
from matplotlib import pyplot as plt

img = cv.imread('simple.jpg',0)

# Initiate FAST object with default values
fast = cv.FastFeatureDetector_create()

# find and draw the keypoints
kp   = fast.detect(img, None)
img2 = cv.drawKeypoints(img, kp, None, color=(255, 0, 0))

# Print all default params
print( "Threshold: {}".format(fast.getThreshold()) )
print( "nonmaxSuppression:{}".format(fast.getNonmaxSuppression()) )
print( "neighborhood: {}".format(fast.getType()) )
print( "Total Keypoints with nonmaxSuppression: {}".format(len(kp)) )
cv.imwrite('fast_true.png',img2)

# Disable nonmaxSuppression
fast.setNonmaxSuppression(0)
kp = fast.detect(img, None)
print( "Total Keypoints without nonmaxSuppression: {}".format(len(kp)) )
img3 = cv.drawKeypoints(img, kp, None, color=(0,0,255))
cv.imwrite('fast_false.png',img3)

plt.subplot(121), plt.imshow(img2)
plt.subplot(122), plt.imshow(img3)
plt.show()

Threshold: 10
nonmaxSuppression:True
neighborhood: 2
Total Keypoints with nonmaxSuppression: 431
Total Keypoints without nonmaxSuppression: 1575


<Figure size 640x480 with 2 Axes>

See the results. First image shows FAST with nonmaxSuppression and second one without nonmaxSuppression:

#### Additional Resources

1. Edward Rosten and Tom Drummond, “Machine learning for high speed corner detection” in 9th European Conference on Computer Vision, vol. 1, 2006, pp. 430–443.
2. Edward Rosten, Reid Porter, and Tom Drummond, "Faster and better: a machine learning approach to corner detection" in IEEE Trans. Pattern Analysis and Machine Intelligence, 2010, vol 32, pp. 105-119.

#### Exercises